# Cleanup

After building completing the notebooks you may want to delete the following to prevent any unwanted charges:

* Forecast Exports
* Forecasts
* Predictors
* Datasets
* Dataset Groups

The code snippets below will cover the base use case of creating items in notebooks 1 - 3. You can expand upon this to delete content created in other notebooks.

## Imports and Connecting to AWS

The following lines import all the necessary libraries and then connect you to Amazon Forecast.


In [28]:
import sys
import os
import json
import time
import boto3
import pandas as pd
import botocore.exceptions

The line below will retrieve your shared variables from the earlier notebooks.

In [29]:
%store -r

Once again connect to the Forecast APIs via the SDK.

In [30]:
session = boto3.Session(region_name=region) 
forecast = session.client(service_name='forecast') 
forecastquery = session.client(service_name='forecastquery')

## Defining the Things to Cleanup

In the previous notebooks you stored several variables at the end of each, now that they have been retrieved above, the cells below will delete the items that were created one at a time until all items that were created have been removed.

In [31]:
def wait_till_delete(callback, check_time = 5, timeout = None):

    elapsed_time = 0
    while timeout is None or elapsed_time < timeout:
        try:
            out = callback()
        except botocore.exceptions.ClientError as e:
            # When given the resource not found exception, deletion has occured
            if e.response['Error']['Code'] == 'ResourceNotFoundException':
                print('Successful delete')
                return
            else:
                raise
        time.sleep(check_time)  # units of seconds
        elapsed_time += check_time

    raise TimeoutError( "Forecast resource deletion timed-out." )

In [32]:
# Delete the Exports
exports = forecast.list_forecast_export_jobs()
exports = [armia_export_arn, prophet_export_arn, deep_ar_export_arn]
for job in exports:
    forecast.delete_forecast_export_job(ForecastExportJobArn=job)

In [ ]:
# Delete the Foreacst:
wait_till_delete(lambda: forecast.delete_forecast(ForecastArn=prophet_forecast_arn))
wait_till_delete(lambda: forecast.delete_forecast(ForecastArn=arima_forecast_arn))
wait_till_delete(lambda: forecast.delete_forecast(ForecastArn=deeparp_forecast_arn))

In [ ]:
# Delete the Predictor:
wait_till_delete(lambda: forecast.delete_predictor(PredictorArn=arima_arn))
wait_till_delete(lambda: forecast.delete_predictor(PredictorArn=prophet_arn))
wait_till_delete(lambda: forecast.delete_predictor(PredictorArn=deeparp_arn))

In [ ]:
# Delete Import
wait_till_delete(lambda: forecast.delete_dataset_import_job(DatasetImportJobArn=ds_import_job_arn))

In [ ]:
# Delete the Dataset:
wait_till_delete(lambda: forecast.delete_dataset(DatasetArn=target_datasetArn))

In [ ]:
# Delete the DatasetGroup:
wait_till_delete(lambda: forecast.delete_dataset_group(DatasetGroupArn=datasetGroupArn))

In [21]:
# Delete your file in S3
s3 = boto3.resource('s3')
bucket = s3.Bucket(bucket_name)
bucket.objects.all().delete()
# Delete the bucket
bucket.delete()


{'ResponseMetadata': {'RequestId': '7615E9786514E404',
  'HostId': '2lor2kQmQzppvchJpQ6YGtv8pUDnQ57qpgqSENNovTkumMpqeBVmTqbLWJuJyDWgejjgg1MQy6E=',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'x-amz-id-2': '2lor2kQmQzppvchJpQ6YGtv8pUDnQ57qpgqSENNovTkumMpqeBVmTqbLWJuJyDWgejjgg1MQy6E=',
   'x-amz-request-id': '7615E9786514E404',
   'date': 'Sat, 25 Apr 2020 20:28:06 GMT',
   'server': 'AmazonS3'},
  'RetryAttempts': 0}}

## IAM Policy Cleanup

The very last step in the notebooks is to remove the policies that were attached to a role and then to delete it. No changes should need to be made here, just execute the cell.

In [23]:
# IAM policies should also be removed
iam = boto3.client("iam")

iam.delete_role(RoleName=role_name)

{'ResponseMetadata': {'RequestId': '09ef27f3-3028-4cf6-bd1c-9ff93e8c3cac',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '09ef27f3-3028-4cf6-bd1c-9ff93e8c3cac',
   'content-type': 'text/xml',
   'content-length': '200',
   'date': 'Sat, 25 Apr 2020 20:28:18 GMT'},
  'RetryAttempts': 0}}

All that remains to cleanup here is to now go back to the CloudFormation console and delete the stack. You have successfully removed all resources that were created.